Training on Shakespeare data

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-29 22:26:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.26MB/s    in 0.2s    

2025-12-29 22:26:56 (6.26 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [48]:
import pandas as pd
import torch

In [49]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [10]:
print(len(text))

1115394


In [11]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [13]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [25]:
# making the string/int to index
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

# just testing
# print(stoi[' '])
# print(itos[10])

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
# testing encode and decode
print(encode("Tamim"))
print(decode([32, 39, 51, 47, 51]))

[32, 39, 51, 47, 51]
Tamim


In [35]:
def build_vocab(text):
    chars = sorted(list(set(text)))
    stoi = {ch:i for i,ch in enumerate(chars)}
    itos = {i:ch for i,ch in enumerate(chars)}
    return stoi, itos

Create encoding and decoding

In [45]:
def encoder(text, stoi):
  return [stoi[ch] for ch in text]

def decoder(indices, itos):
    return ''.join([itos[i] for i in indices])

In [47]:
stoi, itos = build_vocab(text)

ecoded = encoder("Tamim", stoi)
decoded = decoder(ecoded, itos)

print(ecoded)
print(decoded)

[32, 39, 51, 47, 51]
Tamim


Encode the entire text file

In [54]:
# here we are using torch.tensor

data = torch.tensor(encoder(text, stoi), dtype=torch.long)
print(data.dtype, data.shape)
# first 100 characters
print(data[:100])

torch.int64 torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Split data

In [55]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [59]:
block_size = 8
# train_data[:block_size + 1]

In [64]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  print("Current Context", context)
  target = y[t]
  print("Current Target", target)

Current Context tensor([18])
Current Target tensor(47)
Current Context tensor([18, 47])
Current Target tensor(56)
Current Context tensor([18, 47, 56])
Current Target tensor(57)
Current Context tensor([18, 47, 56, 57])
Current Target tensor(58)
Current Context tensor([18, 47, 56, 57, 58])
Current Target tensor(1)
Current Context tensor([18, 47, 56, 57, 58,  1])
Current Target tensor(15)
Current Context tensor([18, 47, 56, 57, 58,  1, 15])
Current Target tensor(47)
Current Context tensor([18, 47, 56, 57, 58,  1, 15, 47])
Current Target tensor(58)


In [71]:
torch.manual_seed(1337)
batch_size = 4

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('input -->', xb.shape)
print('target -->', yb.shape)
print('___' * 20)
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"input: {context.tolist()} target: {target}")

input --> torch.Size([4, 8])
target --> torch.Size([4, 8])
____________________________________________________________
input: [24] target: 43
input: [24, 43] target: 58
input: [24, 43, 58] target: 5
input: [24, 43, 58, 5] target: 57
input: [24, 43, 58, 5, 57] target: 1
input: [24, 43, 58, 5, 57, 1] target: 46
input: [24, 43, 58, 5, 57, 1, 46] target: 43
input: [24, 43, 58, 5, 57, 1, 46, 43] target: 39
input: [44] target: 53
input: [44, 53] target: 56
input: [44, 53, 56] target: 1
input: [44, 53, 56, 1] target: 58
input: [44, 53, 56, 1, 58] target: 46
input: [44, 53, 56, 1, 58, 46] target: 39
input: [44, 53, 56, 1, 58, 46, 39] target: 58
input: [44, 53, 56, 1, 58, 46, 39, 58] target: 1
input: [52] target: 58
input: [52, 58] target: 1
input: [52, 58, 1] target: 58
input: [52, 58, 1, 58] target: 46
input: [52, 58, 1, 58, 46] target: 39
input: [52, 58, 1, 58, 46, 39] target: 58
input: [52, 58, 1, 58, 46, 39, 58] target: 1
input: [52, 58, 1, 58, 46, 39, 58, 1] target: 46
input: [25] target

In [72]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [88]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # bigram: embedding from vocab_size -> vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, C)
        B, T, C = logits.shape
        loss = None
        if targets is not None:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:, -1, :]             # focus on last time step
            probs = F.softmax(logits, dim=-1)     # convert to probabilities
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


m = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)
print("logits shape:", logits.shape)
print("loss:", loss.item())

# Generate sequence
start_idx = torch.zeros((1,1), dtype=torch.long)
gen_idx = m.generate(start_idx, max_new_tokens=100)

# Decode generated indices
decoded_text = decoder(gen_idx[0].tolist(), itos)
print("generated text:", decoded_text)


logits shape: torch.Size([32, 65])
loss: 4.878634929656982
generated text: 
SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [89]:
optimzer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [94]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimzer.zero_grad(set_to_none=True)
  loss.backward()
  optimzer.step()
print(loss.item())

2.481635332107544


In [97]:
start_idx = torch.zeros((1,1), dtype=torch.long)
gen_idx = m.generate(start_idx, max_new_tokens=300)

# Decode generated indices
decoded_text = decoder(gen_idx[0].tolist(), itos)
print("generated text:", decoded_text)

generated text: 


NDWARDWheore odesomyoumo ind asqu thever?

'serire w f meat arnd my, aye.
ANGARI'Thatharot dadis fe powe bl omoforrashe ixppr fis th selld thit b.
Ame hyout
HAgenether is be itrugaporsat u! wime bes e:
Ma m by an l wha in


AT:
rticofte'd f ghetocerd dathey IZAthiss, LUTy matsurorcker--beage Lourd


In [98]:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

Using math we make token to talk to each other

In [101]:
xbow = torch.zeros(B, T, C)
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1]
    xbow[b, t] = torch.mean(xprev, 0)

In [102]:
x[0]

tensor([[ 0.9962, -0.6249],
        [-0.9852,  1.7089],
        [ 2.2263,  1.1026],
        [-1.8915, -1.4591],
        [ 0.3113, -0.0865],
        [-1.0957,  1.0245],
        [ 1.3180,  0.8325],
        [-0.4279,  0.2963]])

Here is the math trick to make it more efficient than above

In [112]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

True

In [120]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)

In [121]:
#Better Version
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

Using softmax trick in mat we can make it better

In [124]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [108]:
# torch.manual_seed(42)
# a = torch.ones(3, 3)
# a = a / torch.sum(a, 1, keepdim=True)
# b = torch.randint(0,10, (3,2)).float()
# c = a @ b

In [128]:
torch.manual_seed(1337)

B, T, C = 4, 8, 32
x = torch.zeros(B, T, C)
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v
# out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [129]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
         [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
         [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1667, 0.166

In [130]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [134]:
k.var()

tensor(0.9182)

In [135]:
q.var()

tensor(0.9487)

In [136]:
wei.var()

tensor(0.8471)

In [137]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [139]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [140]:
class BatchNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
# -*- coding: utf-8 -*-
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

# Set initial hyperparameters and device
batch_size = 64
torch.manual_seed(1337)
max_iteration = 5000
block_size = 256
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
learning_rate = 1e-3
eval_internal = 500
n_embed = 384
dropout = 0.2
n_head = 6
n_layer = 6

# Load input data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


# Build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

# Index mappings
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def build_vocab(text):
    chars = sorted(list(set(text)))
    stoi = {ch: i for i, ch in enumerate(chars)}
    itos = {i: ch for i, ch in enumerate(chars)}
    return stoi, itos

# Encoding and decoding functions
def encoder(text, stoi):
    return [stoi[ch] for ch in text]

def decoder(indices, itos):
    return ''.join([itos[i] for i in indices])

stoi, itos = build_vocab(text)

ecoded = encoder("Tamim", stoi)
decoded = decoder(ecoded, itos)
print(ecoded)
print(decoded)

# Encode the entire text file
data = torch.tensor(encoder(text, stoi), dtype=torch.long)
print(data.dtype, data.shape)
print(data[:100])  # First 100 characters

# Split data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Demo: show context-target pairs for understanding
demo_block = 8
x = train_data[:demo_block]
y = train_data[1:demo_block + 1]

for t in range(demo_block):
    context = x[:t + 1]
    print("Current Context", context)
    target = y[t]
    print("Current Target", target)

torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print('input -->', xb.shape)
print('target -->', yb.shape)
print('___' * 20)
# Demo: show a few examples from first batch
for b in range(min(2, batch_size)):
    for t in range(min(8, block_size)):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"input: {context.tolist()} target: {target}")

# Self-attention head
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5  # scale by head_size, not C
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        return wei @ v

# Multi-head attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.GELU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        # bigram: embedding from vocab_size -> vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)  # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb #(b, t, n_embed)

        x = self.blocks(x) # apply transformer blocks
        x = self.ln_f(x)   # apply final layer norm
        logits = self.lm_head(x)  #(b, t, vocab_size)


        if targets is None:
            loss = None

        else:
            B, T, C= logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) where each element is an integer representing a character
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] # get the last block_size characters
            logits, _ = self(idx_cond) # get the logits for the next character
            logits = logits[:, -1, :] # get the logits for the last character
            probs = F.softmax(logits, dim=-1) # convert the logits to probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # sample the next character
            idx = torch.cat((idx, idx_next), dim=1) # add the next character to the context
        return idx

model = BigramLanguageModel()
m = model.to(device)

# Print model size
print(f"Model parameters: {sum(p.numel() for p in m.parameters())/1e6:.2f}M")

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iteration):

    if iter % eval_internal == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

start_idx = torch.zeros((1, 1), dtype=torch.long, device=device)
gen_idx = m.generate(start_idx, max_new_tokens=500)

# Decode generated indices
generated_text = decoder(gen_idx[0].tolist(), itos)
print("generated text:", generated_text)


--2025-12-30 01:51:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-12-30 01:51:07 (43.4 MB/s) - ‘input.txt’ saved [1115394/1115394]


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[32, 39, 51, 47, 51]
Tamim
torch.int64 torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,